In [17]:
from yucca.image_processing.transforms.Spatial import Spatial
from yucca.image_processing.transforms.BiasField import BiasField
from yucca.image_processing.transforms.Noise import AdditiveNoise, MultiplicativeNoise
from yucca.image_processing.transforms.Ghosting import MotionGhosting
from yucca.image_processing.transforms.Ringing import GibbsRinging
from nibabel.testing import data_path
import nibabel as nib
import os
from copy import deepcopy
import numpy as np
import timeit

np.random.seed(4215235)
iterations = 2
# We use a publicly available sample from Nibabel
example_file = os.path.join(data_path, 'example4d.nii.gz')
im = nib.load(example_file)
imarr = im.get_fdata()[:, :, :, 0]

# If you want to use a local example for dataset specific times, 
# uncomment the following and supply the path to a local image the imarr variable

#local_file = 'YOUR/PATH/HERE'
#im = nib.load(local_file)
#imarr = im.get_fdata()

# Convert it to the format expected of the transforms
# a dict of {"image": image, "seg": segmentation}
# with samples for dims (b, c, x, y, z) for 3D or (b, c, x, y) for 2D
# In this case we use slice 12 for an image of (128, 96, 24) - adapt this as you please.
print("3D shape: ", imarr.shape)
imarr = imarr[np.newaxis, np.newaxis, :, :, 12]
imarr = np.vstack([imarr]*50)
seg = np.zeros(imarr.shape)
datadict = {"image": imarr, "label": seg}
print("2D shape: ", imarr.shape[2:])
print("2D shape with batch and channel dimension: ", imarr.shape)

3D shape:  (128, 96, 24)
2D shape:  (128, 96)
2D shape with batch and channel dimension:  (50, 1, 128, 96)


In [18]:
idx = np.array(imarr.shape[2:])//4
imarr_half = imarr[:, :, idx[0]:-idx[0], idx[1]:-idx[1]]
seg_half = np.zeros(imarr_half.shape)
datadict_half = {"image": imarr_half, "label": seg_half}

In [19]:
# Applying Rotation
tform = Spatial(x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45))
print("Rotation: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)

Rotation:  0.07687035000003561 
  Rotation half:  0.019398150000142778


In [21]:
# Applying Deformation
from yucca.image_processing.transforms.Spatial import Spatial

tform = Spatial(deform_alpha=(700, 700),
                deform_sigma=(10, 10))
print("Deformation: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Deformation half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Deformation:  0.07694740000010825 
  Deformation half:  0.019714600000042992


In [25]:
# Applying Scaling
tform = Spatial(scale_factor=(1.5, 1.5))
print("Scaling: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Scaling half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)

Scaling:  0.07160910000015974 
  Scaling half:  0.01951615000007223


In [26]:
# Applying Rotation, Deformation and Scaling WITHOUT cropping
tform = Spatial(x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
                scale_factor=(1.5, 1.5))
print("Rotation, Deformation and Scaling WITHOUT cropping: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation, Deformation and Scaling WITHOUT cropping half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Rotation, Deformation and Scaling WITHOUT cropping:  0.057532350000201404 
  Rotation, Deformation and Scaling WITHOUT cropping half:  0.015599049999991621


In [27]:
# Applying Rotation, Deformation and Scaling WITH cropping
tform = Spatial(crop=True,
                patch_size=np.array(imarr.shape[2:])//2,
                x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
               scale_factor=(1.5, 1.5))
# Applying Rotation, Deformation and Scaling WITH cropping
tform2 = Spatial(crop=True,
                patch_size=np.array(imarr_half.shape[2:])//2,
                x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
                scale_factor=(1.5, 1.5))
print("Rotation, Deformation and Scaling WITH cropping: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation, Deformation and Scaling WITH cropping half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Rotation, Deformation and Scaling WITH cropping:  0.032720100000005914 
  Rotation, Deformation and Scaling WITH cropping half:  0.021289699999897493


In [28]:
# Applying Additive Noise
tform = AdditiveNoise(p_per_sample=1, mean=(0, 0),
                      sigma=(25, 25))
print("Additive Noise: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Additive Noise half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Additive Noise:  0.019864849999976286 
  Additive Noise half:  0.00764219999996385


In [29]:

# Applying Multiplicative Noise
tform = MultiplicativeNoise(p_per_sample=1, mean=(0, 0),
                            sigma=(0.1, 0.1))
print("Multiplicative Noise: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Multiplicative Noise half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Multiplicative Noise:  0.025139599999874918 
  Multiplicative Noise half:  0.0059195500000441825


In [30]:
# Applying Bias Field
tform = BiasField(p_per_sample=1)
print("Bias Field: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Bias Field half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Bias Field:  0.046703000000206885 
  Bias Field half:  0.031693199999836


In [31]:
# Applying Motion Ghosting
tform = MotionGhosting(p_per_sample=1, alpha=(0.6, 0.6),
                       numReps=(3, 4),
                       axes=(0, 1))
print("Motion Ghosting: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Motion Ghosting half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Motion Ghosting:  0.029291249999914726 
  Motion Ghosting half:  0.010056399999939458


In [32]:

# Applying Gibbs Ringing
tform = GibbsRinging(p_per_sample=1, cutFreq=(30, 36),
                     axes=(0, 1))
print("Gibbs Ringing: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Gibbs Ringing half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Gibbs Ringing:  0.035132399999838526 
  Gibbs Ringing half:  0.012108300000136296
